In [2]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def census(target,lis):
    answer=0
    for n in lis:
        if n==target:answer=answer+1
    return answer

def positions(target,lst):
    ln=len(lst)
    answer=[]
    for j in [0..ln-1]:
        entry=lst[j]
        if entry==target:answer=answer+[j+1]
    return(answer)

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [39]:
ps=positions(0,od);print(ps) #A037453 Positive numbers whose base-5 representation contains no 3 or 4.

[1, 2, 5, 6, 7, 10, 11, 12, 25, 26, 27, 30, 31, 32, 35, 36, 37, 50, 51, 52, 55, 56, 57, 60, 61, 62, 125, 126, 127, 130, 131, 132, 135, 136, 137, 150, 151, 152, 155, 156, 157, 160, 161, 162, 175, 176, 177, 180, 181, 182, 185, 186, 187]


In [3]:
# verify narrow version, p = 7
import time

def p7RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    for a in [1..n+1]:
        term=rmnjntau(a)
        ans=ans+term*x^a
    return D(ans)

def recipP7(n):
    ans=L(1/p7RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p7RecipPowerConstantTerm(k):
    g=recipP7(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

no=[]
zeros=[]
od=[]
start=time.time()
for a in [1..20]: #default precision must be > max a.
    c=p7RecipPowerConstantTerm(a)
    o7=order(7,c)
    if o7==0:zeros=zeros+[a]
    od=od+[o7]
    c5=census(5,a.digits(7))
    c6=census(6,a.digits(7))
    if c5==0:
        if c6==0:
              no=no+[a]
    finish=time.time()
    print((a,finish-start))
print();print("od:");print(od);print()
ps=positions(0,od)
vno=vector(no);vps=vector(ps)
print("vno-vps:");print();print(vno-vps)

(1, 0.016301870346069336)
(2, 0.02754807472229004)
(3, 0.03902888298034668)
(4, 0.05137014389038086)
(5, 0.06168413162231445)
(6, 0.07700300216674805)
(7, 0.09350109100341797)
(8, 0.10787010192871094)
(9, 0.12219405174255371)
(10, 0.1364121437072754)
(11, 0.15051698684692383)
(12, 0.16493988037109375)
(13, 0.17883896827697754)
(14, 0.1934361457824707)
(15, 0.2087719440460205)
(16, 0.22402191162109375)
(17, 0.23960494995117188)
(18, 0.25364112854003906)
(19, 0.2673161029815674)
(20, 0.2813260555267334)

od:
[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 2]

vno-vps:

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


In [4]:
# verify narrow version, p = 7
import time

def p7RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    for a in [1..n+1]:
        term=rmnjntau(a)
        ans=ans+term*x^a
    return D(ans)

def recipP7(n):
    ans=L(1/p7RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p7RecipPowerConstantTerm(k):
    g=recipP7(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

no=[]
zeros=[]
od=[]
start=time.time()
for a in [1..5000]: #default precision must be > max a.
    c=p7RecipPowerConstantTerm(a)
    o7=order(7,c)
    if o7==0:zeros=zeros+[a]
    od=od+[o7]
    c5=census(5,a.digits(7))
    c6=census(6,a.digits(7))
    if c5==0:
        if c6==0:
              no=no+[a]
    finish=time.time()
    print((a,finish-start))
print();print("od:");print(od);print()
ps=positions(0,od)
vno=vector(no);vps=vector(ps)
print("vno-vps:");print();print(vno-vps)

(1, 0.010297060012817383)
(2, 0.019703149795532227)
(3, 0.029798269271850586)
(4, 0.04095792770385742)
(5, 0.05116128921508789)
(6, 0.0651242733001709)
(7, 0.08031225204467773)
(8, 0.0949561595916748)
(9, 0.10976099967956543)
(10, 0.12374210357666016)
(11, 0.13788533210754395)
(12, 0.15237212181091309)
(13, 0.16626214981079102)
(14, 0.22179722785949707)
(15, 0.23583626747131348)
(16, 0.2501649856567383)
(17, 0.26576995849609375)
(18, 0.27979421615600586)
(19, 0.29384708404541016)
(20, 0.30794715881347656)
(21, 0.32424116134643555)
(22, 0.33988308906555176)
(23, 0.3556632995605469)
(24, 0.3719370365142822)
(25, 0.3899972438812256)
(26, 0.4082162380218506)
(27, 0.4274141788482666)
(28, 0.4460451602935791)
(29, 0.4663841724395752)
(30, 0.48548412322998047)
(31, 0.5049641132354736)
(32, 0.5248551368713379)
(33, 0.544471025466919)
(34, 0.5647580623626709)
(35, 0.5855340957641602)
(36, 0.609321117401123)
(37, 0.632317304611206)
(38, 0.6546142101287842)
(39, 0.678184986114502)
(40, 0.70236515

(327, 110.7995662689209)
(328, 111.80003929138184)
(329, 112.80091333389282)
(330, 113.81872630119324)
(331, 114.82644605636597)
(332, 115.85095930099487)
(333, 116.85779929161072)
(334, 117.87367415428162)
(335, 118.87506628036499)
(336, 119.91406416893005)
(337, 120.96681499481201)
(338, 122.02057719230652)
(339, 123.09017515182495)
(340, 124.13983511924744)
(341, 125.18726205825806)
(342, 126.24349427223206)
(343, 127.3072521686554)
(344, 128.3652422428131)
(345, 129.43749928474426)
(346, 130.51346731185913)
(347, 131.57973718643188)
(348, 132.6627812385559)
(349, 133.7536482810974)
(350, 134.84945511817932)
(351, 135.985276222229)
(352, 137.1340982913971)
(353, 138.27948713302612)
(354, 139.44856810569763)
(355, 140.6220932006836)
(356, 141.76548624038696)
(357, 142.92662024497986)
(358, 144.0934190750122)
(359, 145.23949909210205)
(360, 146.41728901863098)
(361, 147.59028100967407)
(362, 148.83511924743652)
(363, 150.05378198623657)
(364, 151.29220032691956)
(365, 152.533715009689

(651, 854.0175361633301)
(652, 857.8951852321625)
(653, 861.7971603870392)
(654, 865.689239025116)
(655, 869.6171283721924)
(656, 873.501216173172)
(657, 877.4161012172699)
(658, 881.3630232810974)
(659, 885.3205461502075)
(660, 889.273490190506)
(661, 893.2468581199646)
(662, 897.2175643444061)
(663, 901.22154712677)
(664, 905.2762780189514)
(665, 909.2904641628265)
(666, 913.3225049972534)
(667, 917.3821842670441)
(668, 921.4565403461456)
(669, 925.5318551063538)
(670, 929.6158893108368)
(671, 933.7468810081482)
(672, 937.8634660243988)
(673, 941.9818203449249)
(674, 946.1002511978149)
(675, 950.2460081577301)
(676, 954.441159248352)
(677, 958.6200811862946)
(678, 962.8166692256927)
(679, 967.0250890254974)
(680, 971.2350950241089)
(681, 975.4725151062012)
(682, 979.7188432216644)
(683, 983.9830241203308)
(684, 988.2404391765594)
(685, 992.5190691947937)
(686, 996.8085501194)
(687, 1001.0869309902191)
(688, 1005.3616812229156)
(689, 1009.6830163002014)
(690, 1014.0427069664001)
(691,

(972, 2793.4322533607483)
(973, 2802.15806722641)
(974, 2810.7443583011627)
(975, 2819.7881932258606)
(976, 2828.56583404541)
(977, 2837.5542562007904)
(978, 2846.3800320625305)
(979, 2855.117061138153)
(980, 2864.009194135666)
(981, 2872.915060043335)
(982, 2881.4826822280884)
(983, 2890.118910074234)
(984, 2898.6739630699158)
(985, 2907.2898240089417)
(986, 2915.9313480854034)
(987, 2924.6126232147217)
(988, 2933.2830452919006)
(989, 2941.9897260665894)
(990, 2950.709342956543)
(991, 2959.426063299179)
(992, 2968.16685628891)
(993, 2976.9402751922607)
(994, 2985.685622215271)
(995, 2994.474603176117)
(996, 3003.338730096817)
(997, 3012.161950111389)
(998, 3021.0360839366913)
(999, 3029.9554471969604)
(1000, 3038.813669204712)
(1001, 3047.693132162094)
(1002, 3056.559740304947)
(1003, 3065.4901192188263)
(1004, 3074.505998134613)
(1005, 3083.4961111545563)
(1006, 3092.5236101150513)
(1007, 3101.5876982212067)
(1008, 3110.704243183136)
(1009, 3119.789954185486)
(1010, 3128.883764982223

(1287, 6408.720649957657)
(1288, 6423.445315122604)
(1289, 6438.30896115303)
(1290, 6453.154319286346)
(1291, 6467.908016204834)
(1292, 6482.763743162155)
(1293, 6497.59840631485)
(1294, 6512.508093357086)
(1295, 6527.328784227371)
(1296, 6542.240888118744)
(1297, 6557.1424951553345)
(1298, 6572.063184261322)
(1299, 6587.040867090225)
(1300, 6602.078143119812)
(1301, 6617.172429084778)
(1302, 6632.328908205032)
(1303, 6647.41467499733)
(1304, 6662.444580078125)
(1305, 6677.608043193817)
(1306, 6692.681891202927)
(1307, 6707.830229997635)
(1308, 6723.016872167587)
(1309, 6738.287078142166)
(1310, 6753.596741199493)
(1311, 6768.82488822937)
(1312, 6783.989675045013)
(1313, 6799.305563211441)
(1314, 6814.7257771492)
(1315, 6830.072703123093)
(1316, 6845.507501125336)
(1317, 6860.844786167145)
(1318, 6876.207396268845)
(1319, 6891.603209018707)
(1320, 6907.153438091278)
(1321, 6922.610835075378)
(1322, 6938.164800167084)
(1323, 6953.797665119171)
(1324, 6969.282253026962)
(1325, 6984.82940

(1600, 12387.870309114456)
(1601, 12411.497891187668)
(1602, 12434.577649354935)
(1603, 12458.127029180527)
(1604, 12481.937990188599)
(1605, 12505.933331012726)
(1606, 12530.070415258408)
(1607, 12553.795069217682)
(1608, 12577.674133062363)
(1609, 12601.892320156097)
(1610, 12625.861491203308)
(1611, 12649.588020086288)
(1612, 12673.458998203278)
(1613, 12697.632320165634)
(1614, 12721.719281196594)
(1615, 12745.990056991577)
(1616, 12770.15551328659)
(1617, 12794.177090168)
(1618, 12818.177513122559)
(1619, 12842.612126111984)
(1620, 12867.3523812294)
(1621, 12892.076818943024)
(1622, 12916.427962303162)
(1623, 12941.095842123032)
(1624, 12965.494229078293)
(1625, 12990.07746720314)
(1626, 13014.529759168625)
(1627, 13038.602810144424)
(1628, 13062.954820156097)
(1629, 13087.409263134003)
(1630, 13111.93012714386)
(1631, 13136.706611394882)
(1632, 13161.210233211517)
(1633, 13186.105384349823)
(1634, 13211.490463018417)
(1635, 13235.842190265656)
(1636, 13260.084485292435)
(1637, 13

KeyboardInterrupt: 

In [5]:
print();print("od:");print(od);print()
ps=positions(0,od)
vno=vector(no);vps=vector(ps)
print("vno-vps:");print();print(vno-vps)


od:
[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 3, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 2, 3, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 3, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 4, 2, 3, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 3, 2, 3, 3, 3, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 3, 2, 1, 1, 1, 1, 2, 2, 4, 1, 1, 1, 1, 3, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3

In [ ]:
# verify narrow version, p = 7
import time

def p7RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    for a in [1..n+1]:
        term=a*sigma(a,k=3)
        ans=ans+term*x^a
    return D(ans)

def recipP7(n):
    ans=L(1/p7RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p7RecipPowerConstantTerm(k):
    g=recipP7(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

no=[]
zeros=[]
od=[]
start=time.time()
for a in [1..5000]: #default precision must be > max a.
    c=p7RecipPowerConstantTerm(a)
    o7=order(7,c)
    if o7==0:zeros=zeros+[a]
    od=od+[o7]
    c5=census(5,a.digits(7))
    c6=census(6,a.digits(7))
    if c5==0:
        if c6==0:
              no=no+[a]
    finish=time.time()
    print((a,finish-start))
print();print("od:");print(od);print()
ps=positions(0,od)
vno=vector(no);vps=vector(ps)
print("vno-vps:");print();print(vno-vps)